Import

In [1]:
import sys
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity as ssim



In [2]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, Input
import os
import math

realization of evaluation index function

In [3]:
# define function for peak signal-to-noise ration (PSNR)
def psnr(target, ref) : 
    
    # assume RGB/BGR image
    target_data = target.astype(float)
    ref_data = ref.astype(float)
    
    diff = ref_data - target_data
    diff = diff.flatten('C')
    
    rmse = math.sqrt(np.mean(diff ** 2.))
    
    return 20 * math.log10(255./ rmse)

# define function for mean squared error (MSE)
def mse(target, ref):
    
    # MSE is the sum of the squared fifference between the two the images
    err = np.sum((target.astype('float') - ref.astype('float')) ** 2)
    err /= float(target.shape[0] * target.shape[1])
    
    return err

# define function that combines all three image quality metrics
def compare_images(target, ref):
    scores = []
    scores.append(psnr(target, ref))
    scores.append(mse(target, ref))
    scores.append(ssim(target, ref, multichannel= True))
    
    return scores

prepare degraded images by introducing quality distortions via resizing

In [8]:
def prepare_images(path, factor):
    
    # loop through the file in the directory 
    for file in os.listdir(path):
        
        # open the file
        img = cv2.imread(path + '/' + file)
        
        # find old and new image dimensions
        h, w, c = img.shape
        new_height = int(h / factor)
        new_width = int(w / factor)
        
        # resize the image - down
        img = cv2.resize(img, (new_width, new_height), interpolation = cv2.INTER_LINEAR)
        
        # resize the image - up
        img = cv2.resize(img, (w, h), interpolation = cv2.INTER_LINEAR)
        
        # save the image
        print('Saving {}'.format(file))
        cv2.imwrite('images/{}'.format(file), img)

In [9]:
prepare_images('D:/Personal Project/source/', 2)

Saving baboon.bmp
Saving baby_GT.bmp
Saving barbara.bmp
Saving bird_GT.bmp
Saving bridge.bmp
Saving butterfly_GT.bmp
Saving coastguard.bmp
Saving comic.bmp
Saving face.bmp
Saving flowers.bmp
Saving foreman.bmp
Saving head_GT.bmp
Saving lenna.bmp
Saving man.bmp
Saving monarch.bmp
Saving pepper.bmp
Saving ppt3.bmp
Saving woman_GT.bmp
Saving zebra.bmp


In [10]:
for file in os.listdir('images/'):
    
    # open target and reference images
    target = cv2.imread('images/{}'.format(file))
    ref = cv2.imread('source/{}'.format(file))
    
    # calculate the scores
    scores = compare_images(target, ref)
    
    # print a;; three scores
    print('{}\nPSNR: {}\nSSIM: {}\n'.format(file, scores[0], scores[1], scores[2]))

baboon.bmp
PSNR: 22.157084083442548
SSIM: 1187.1161333333334

baby_GT.bmp
PSNR: 34.37180640966199
SSIM: 71.28874588012695

barbara.bmp
PSNR: 25.906629837568126
SSIM: 500.65508535879627

bird_GT.bmp
PSNR: 32.896644728720005
SSIM: 100.12375819830247

bridge.bmp
PSNR: 25.850528790115554
SSIM: 507.1643714904785

butterfly_GT.bmp
PSNR: 24.782076560337416
SSIM: 648.6254119873047

coastguard.bmp
PSNR: 27.161600663887082
SSIM: 375.00887784090907

comic.bmp
PSNR: 23.799861502225532
SSIM: 813.2338836565096

face.bmp
PSNR: 30.99220650287191
SSIM: 155.23189718546524

flowers.bmp
PSNR: 27.454504805386147
SSIM: 350.55093922651935

foreman.bmp
PSNR: 30.14456532664372
SSIM: 188.6883483270202

head_GT.bmp
PSNR: 31.020502848237534
SSIM: 154.2237755102041

lenna.bmp
PSNR: 31.47349297867539
SSIM: 138.94800567626953

man.bmp
PSNR: 27.22646369798821
SSIM: 369.4496383666992

monarch.bmp
PSNR: 30.196242365288896
SSIM: 186.45643615722656

pepper.bmp
PSNR: 29.88947161686106
SSIM: 200.1033935546875

ppt3.bmp
PSN

Define the SRCNN model

In [14]:
def model():
    
    # define model type
    SRCNN = Sequential()
    
    # add model layers
    SRCNN.add(Conv2D(filters=128, kernel_size=(9, 9), kernel_initializer='glorot_uniform',
                     activation='relu', padding='valid', use_bias=True, input_shape=(None, None, 1)))
    SRCNN.add(Conv2D(filters=64, kernel_size=(3, 3), kernel_initializer='glorot_uniform',
                     activation='relu', padding='same', use_bias=True))
    SRCNN.add(Conv2D(filters=1, kernel_size=(5,5), kernel_initializer='glorot_uniform',
                     activation='linear', padding='valid', use_bias=True))
    
    # compile model
    SRCNN.compile(optimizer='adam', loss='mse', metrics=['mse'])
    
    return SRCNN
    

Define necessary image processing functions

In [15]:
def modcrop(img, scale):
    tmpsz = img.shape
    sz = tmpsz[0:2]
    sz = sz - np.mod(sz, scale)
    img = img[0:sz[0], 1:sz[1]]
    return img

def shave(image, border):
    img = image[border:-border, border:-border]
    return img

define main prediction function

In [16]:
def predict(image_path):
    
    # load the SRCNN model with weights
    srcnn = model()
    srcnn.load_weights('3051crop_weight_200.h5')
    
    # load the degraded and reference images
    path, file = os.path.split(image_path)
    degraded = cv2.imread(image_path)
    ref = cv2.imread('source/{}'.format(file))
    
    # preprocess the image with madcrop
    ref = modcrop(ref, 3)
    degraded = modcrop(degraded, 3)
    
    # convert the image to VCrCb - (srccn trained on Y channel)
    temp = cv2.cvtColor(degraded, cv2.COLOR_BGR2YCrCb)
    
    # create image slice and normalize
    Y = np.zeros((1, temp.shape[0], temp.shape[1], 1), dtype=float)
    Y[0, :, :, 0] = temp[:, :, 0].astype(float) / 255
    
    # perform super-resolution with srcnn
    pre = srcnn.predict(Y, batch_size=1)
    
    # post-process output
    pre *= 255
    pre[pre[:] > 255] = 255
    pre[pre[:] < 0] = 0
    pre = pre.astype(np.uint8)
    
    # copy Y channel back to image and convert to BGR
    temp = shave(temp, 6)
    temp[:, :, 0] = pre[0, :, :, 0]
    output = cv2.cvtColor(temp, cv2.COLOR_YCrCb2BGR) 
    
    # remove border from reference and degraded image
    ref = shave(ref.astype(np.uint8), 6)
    degraded = shave(degraded.astype(np.uint8), 6)
    
    # image quality caluclations 
    scores = []
    scores.append(compare_images(degraded, ref))
    scores.append(compare_images(output, ref))
    
    # return images and scores
    return ref, degraded, output, scores  
       

In [55]:
ref, degraded, output, scores = predict('images/flowers.bmp')

# print all scores for all images
print('Degraded Image : \nPSNR: {}\nMSE: {}\nSSIM: {}\n'.format(scores[0][0], scores[0][1], scores[0][2]))
print('Reconstructed Image : \nPSNR: {}\nMSE: {}\nSSIM: {}\n'.format(scores[1][0], scores[1][1], scores[1][2]))

# display images as subplots
fig, axs = plt.subplots(1, 3, figsize=(20,8))
axs[0].imshow(cv2.cvtColor(ref, cv2.COLOR_BGR2RGB))
axs[0].set_title("Original")
axs[1].imshow(cv2.cvtColor(degraded, cv2.COLOR_BGR2RGB))
axs[1].set_title("Degraded")
axs[2].imshow(cv2.cvtColor(output, cv2.COLOR_BGR2RGB))
axs[2].set_title("SRCNN")

# remove the x and y tick marks
for ax in axs:
    ax.set_xticks([])
    ax.set_yticks([])

ValueError: could not broadcast input array from shape (356,493) into shape (348,485)